In [17]:
# !pip install geopy
# !pip install folium
# !pip install sklearn
# !pip install matplotlib
# !pip install geopandas
# !pip install urllib
# !pip install html5lib
# !pip install bs4
import html5lib
from bs4 import BeautifulSoup
import urllib, geopy.geocoders, folium, json, requests, io
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
import pandas as pd 
import numpy as np

print('All required libraries successfully imported')


All required libraries successfully imported


In [18]:
CLIENT_ID = 'UATKCT2MMK3S1ZP11HH3BR0YJ42W1ELEZMEVXD0PLWGGAPFO' # your Foursquare ID
CLIENT_SECRET = 'HTVNXNE1GPEFQ4RJKPX3BIAR1BYG3WYQQ3KZSFUFF2MLBRHN' # your Foursquare Secret
ACCESS_TOKEN = 'M4LRXZBVTKQ3KORATCGE5SLAU4UXMZM0DTMSBPYINLG5BLVD' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
radius=500


In [19]:
address = 'Boston, MA'
geolocator = geopy.Nominatim(user_agent="BOS_Diner")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Goelocation of Boston is {}, {}.'.format(latitude,longitude))

Goelocation of Boston is 42.3602534, -71.0582912.


In [20]:
url='https://www.mapsofworld.com/usa/states/massachusetts/lat-long.html'
df_bos=pd.read_html(url,flavor='html5lib')
df_bos=pd.DataFrame(df_bos[2])
df_bos=df_bos.iloc[2:].reset_index(drop=True)
df_bos.columns=['Location','Latitude','Longitude']
df_bos['Location'] = df_bos['Location'].str.replace(' city', '')


url2='https://en.wikipedia.org/wiki/List_of_villages_in_Massachusetts'
df_cty=pd.read_html(url2,flavor='html5lib')
df_cty=pd.DataFrame(df_cty[2])

df_cty=df_cty.drop(columns=['Named place','Notes'])

df_loc= pd.merge(left=df_bos, right=df_cty, left_on='Location', right_on='Municipality')
df_loc=df_loc.drop_duplicates().drop(columns=['Municipality'])
df_loc=df_loc.reset_index(drop=True)

pd.set_option('display.max_rows', None)
df_loc=df_loc[~df_loc.County.str.contains("P.O.")]
df_loc=df_loc.reset_index(drop=True)

map_bos = folium.Map(location=[latitude, longitude], zoom_start=9.5)

for lat, lng, location,county in zip(df_loc['Latitude'], df_loc['Longitude'], df_loc['Location'],df_loc['County']):
    label = '{},{}'.format(location,county)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bos)  

map_bos

# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     borough_latitude, 
#     borough_longitude, 
#     radius, 
#     LIMIT)
# url
# results = requests.get(url).json()



In [21]:
url3='http://marketstatsreports.showingtime.com/MAR_z4jps/sst/202102/00sf.htm#RANGE!A1'
df_price=pd.read_html(url3,flavor='html5lib')
df_price=pd.DataFrame(df_price[0])
df_price=df_price.drop(columns=[0,2,3,5,6,7,8,9,10,11,12,13,14,15,16])
df_price=df_price[5:]
df_price=df_price.drop([355,356,357,358,359,360,361,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404])
df_price=df_price.drop(1, axis=1).join(df_price[1].str.split(' / ', expand=True).stack().reset_index(level=1, drop=True).rename(1))
df_price=df_price.rename(columns={4:'Median Home Price'}).reset_index(drop=True)

df_house= pd.merge(left=df_loc, right=df_price, left_on='Location', right_on=1)
df_house=df_house.drop(columns=[1]).reset_index(drop=True)

df_2br=pd.read_csv("2BR_Rent.csv")
count=list(df_2br['Town'])
strip_count=[]

for i in count:
     strip_count.append(i.rsplit(' ',1)[0])
        
df_2br=df_2br.assign(Town=strip_count)
df= pd.merge(left=df_house, right=df_2br, left_on='Location', right_on='Town')
df=df.drop(columns=['Town']).reset_index(drop=True)
cols=['Location','County','Median Home Price','Median 2BR Rent','Latitude','Longitude']
df=df[cols]
df=df.rename(columns={'Location':'Town/City'})
df[["Latitude", "Longitude"]] = df[["Latitude", "Longitude"]].apply(pd.to_numeric)

In [22]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   res_mi=res*0.621371
   return np.round(res_mi, 2)

In [23]:
def mon_mortgage(P,r,n):
    M = P*(r*(1+r)**n)/((1+r)**n-1)
    return np.round(M,2)

In [28]:
df

,Town/City,County,Median Home Price,Median 2BR Rent,Latitude,Longitude,Monthly Commute Cost
0,Abington,Plymouth,"$530,000",1365.0,42.12,-70.96,139.95
1,Adams,Berkshire,"$191,700",1000.0,42.63,-73.12,860.93
2,Andover,Essex,"$930,000",1187.0,42.66,-71.14,170.46
3,Arlington,Middlesex,"$687,000",1740.0,42.42,-71.16,53.51
4,Athol,Worcester,"$190,150",922.0,42.59,-72.23,498.71
5,Attleboro,Bristol,"$430,000",1014.0,41.93,-71.29,258.35
6,Ayer,Middlesex,"$415,000",1740.0,42.56,-71.58,241.81
7,Barre,Worcester,"$225,000",1192.0,42.42,-72.11,434.46
8,Belchertown,Hampshire,"$325,000",1117.0,42.27,-72.41,559.64
9,Bellingham,Norfolk,"$364,000",1740.0,42.09,-71.48,230.27


In [34]:
df[df.columns[2:4]] = df[df.columns[2:4]].replace('[\$,]', '', regex=True).astype(int)

In [35]:
df

,Town/City,County,Median Home Price,Median 2BR Rent,Latitude,Longitude,Monthly Commute Cost
0,Abington,Plymouth,530000,1365,42.12,-70.96,139.95
1,Adams,Berkshire,191700,1000,42.63,-73.12,860.93
2,Andover,Essex,930000,1187,42.66,-71.14,170.46
3,Arlington,Middlesex,687000,1740,42.42,-71.16,53.51
4,Athol,Worcester,190150,922,42.59,-72.23,498.71
5,Attleboro,Bristol,430000,1014,41.93,-71.29,258.35
6,Ayer,Middlesex,415000,1740,42.56,-71.58,241.81
7,Barre,Worcester,225000,1192,42.42,-72.11,434.46
8,Belchertown,Hampshire,325000,1117,42.27,-72.41,559.64
9,Bellingham,Norfolk,364000,1740,42.09,-71.48,230.27


In [36]:
dist=[]
cost=[]

ppg=3.00
mpg=16.2
wdays=261.5

for i in range(0,len(df)):
    cost.append(round(((haversine_distance(df['Latitude'][i],df['Longitude'][i],latitude,longitude)*2) * (wdays / 12) / mpg) * ppg,2))

df['Monthly Commute Cost']=cost

r=(3/12)/100
n=30*12

mhouse=[]
for i in range(0,len(df)):
    mhouse.append(mon_mortgage(df['Median Home Price'][i],r,n))

df['Monthly Home Payment']=mhouse

In [42]:
map_bos = folium.Map(location=[latitude, longitude], zoom_start=9.5)

folium.CircleMarker(
        [latitude, longitude],
        radius=11,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_bos)
for lat, lng, location, county, medhome, med2br, commute in zip(df['Latitude'], df['Longitude'], df['Town/City'],df_loc['County'],df['Monthly Home Payment'],df['Median 2BR Rent'],df['Monthly Commute Cost']):
    label = '{},{} Home: {} Apt: {} Commute: {} '.format(location,county,medhome,med2br,commute)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bos)  

map_bos


In [ ]:
df